# ChatGPT API 연습

In [1]:
import os
import re
import openai
import markdown
from getpass import getpass

In [7]:
OPENAI_API_KEY = getpass('your API Key(starting with sk-)')

your API Key(starting with sk-)········


In [8]:
TEMPERATURE = 0.1
MAX_TOKENS = 4096  # GPT-3.5의 최대 토큰 길이임(입력, 출력 동일함)
MODEL = 'gpt-3.5-turbo-0613'
openai.api_key = OPENAI_API_KEY
context = []

## 대화형 인공지능의 예시

In [9]:
def check_tokens(items):
    cnt = 0

    if items is None:
        return cnt

    for item in items:
        cnt += len(item['content'])

    return cnt

def conversate():
    while(1):
        message = input('대화:')
        message = message.strip()
    
        if message == '':
            print('대화 내용을 입력하세요.')
            continue
        elif message == 'exit':
            break
    
        # 대화 맥락을 고려하여 전체 최대 토큰을 초과하는지 체크하도록 한다.
        total_cnt = check_tokens(context) + len(message)

        if total_cnt >= MAX_TOKENS:
            context.clear()
            print('context cleared.')

        # ChatGPT 대화를 위한 메시지 형태 설정하기
        if len(context) == 0:
            context.append({"role": "system", "content": "You are a helpful assistant."})
            context.append({"role": "user", "content": message})
        else:
            context.append({"role": "user", "content": message})

        response = openai.ChatCompletion.create(model=MODEL, messages=context, temperature=TEMPERATURE)

        answer = response['choices'][0]['message']['content']
        print(f"AI: {answer}")
        #codes = markdown.markdown(answer, extensions=['fenced_code', 'codehilite'])
        context.append({'role': 'assistant', 'content': answer})

        if check_tokens(context) >= MAX_TOKENS:
            context.clear()

In [10]:
conversate()

대화:안녕
AI: 안녕하세요! 도움이 필요하신가요?
대화:반가워
AI: 반가워요! 저는 여러분을 도와드릴 준비가 되어있습니다. 무엇을 도와드릴까요?
대화:exit


## 논증 분석을 위한 ChatGPT API 활용

In [11]:
query = '''"보기"는 학생이 작성한 글이야. 툴민의 논증(Toulmin's Argumentation Pattern)에 따라 글에 포함되어 있는 주장(claim), 반박(rebuttal), 자료(data), 보장(warrant), 뒷받침(backing), 제한 조건(qualifer)이 명시적으로 포함되어 있는지 확인하고 추출해. 
만약 관련된 요소가 글 속에 포함되어 있지 않다면 '없음'으로 표시해. 
논증 요소의 정의는 다음과 같아.
주장(claim): Assertions about what exists or values that people hold. 
자료(data): Statements that are used as evidence to support the claim. 
보장(warrant): Statements that explain the relationship of the data to the claim. 
제한 조건(qualifier): Special conditions under which the claim holds true. 
뒷받침(backing): Underlying assumptions that are often not made explicit. 
반박(rebuttal): Statements that contradict either the data, warrant, backing or qualifier of an argument. 
또한 학생의 논증에 대한 점수을 0점에서 5점까지로 평가하려고 해. 평가한 결과에 대한 점수와 함께 글을 더 잘 쓸 수 있도록 쉽고 구체적으로 피드백을 줘.
채점 기준은 다음과 같아.
0점: Argumentation does not include anything of elements.
1점: Argumentation consists of arguments that are a simple claim versus a counter-claim or a claim versus a claim.
2점: Argumentation has arguments consisting of a claim versus a claim with either data, warrants, or backings but do not contain any rebuttals.
3점: Argumentation has arguments with a series of claims or counterclaims with either data, warrants, or backings with the occasional weak rebuttal.
4점: Argumentation shows arguments with a claim with a clearly identifiable rebuttal. Such an argument may have several claims and counterclaims as well.
5점: Argumentation displays an extended argument with more than one rebuttal.
수행한 결과를 다음과 같은 형식으로 한글로 반환해 줘.
주장:{결과}
반박:{결과}
자료:{결과}
보장:{결과}
뒷받침:{결과}
제한 조건:{결과}
점수:{결과}
피드백:{결과}
보기:'''

In [12]:
elements = ['주장', '반박', '자료', '보장', '뒷받침', '제한 조건', '점수', '피드백']
elements2 = ['claim', 'rebuttal', 'data', 'warrant', 'back', 'qualify', 'score', 'feedback']
none_list = ['없습니다.', '없음.', '없습니다', '없음', '(없음)', '(없음.)', '(없음).', '(없습니다.)', '(없습니다)']

def inspect_data(text):
    is_first = True
    
    try:
        text = text.strip()
        print('Original:', text)
        query_msg = query + text

        # 메시지 설정하기
        messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": query_msg}
        ]

        # ChatGPT API 호출하기
        response = openai.ChatCompletion.create(model=MODEL, messages=messages, temperature=TEMPERATURE)
        answer = response.choices[0].message['content']
        answer = answer.strip()

        print(answer)

        arguments = {}

        for item in elements:
            arguments[item] = ''

        current_key = None

        for line in answer.split('\n'):
            line = line.strip()
            line = line.strip('\n')

            if line:
                if ':' in line:
                    key, value = line.split(':', 1)
                    current_key = key.strip()
                    value = value.strip()
                    value = value.strip('\n')
                    arguments[current_key] = value
                else:
                    arguments[current_key] += ' ' + line.strip()

        for key, value in arguments.items():
            for item in none_list:
                if item in value:
                    arguments[key] = ''
                    break

        updated_arguments = {}
        for key, value in arguments.items():
            for item in elements:
                if item in key:
                    new_key = item
                    break
                else:
                    new_key = key

            updated_arguments[new_key] = value

        cleaned_arguments = {key.strip("[]"): value for key, value in updated_arguments.items()}
        if cleaned_arguments['점수'] == '':
            cleaned_arguments['점수'] = 0
        else:
            cleaned_arguments['점수'] = int(re.sub(r'\D', '', cleaned_arguments['점수']))

        for i in range(len(elements)):
            cleaned_arguments[elements2[i]] = cleaned_arguments.pop(elements[i])

    except openai.error.APIError as e:
        #Handle API error here, e.g. retry or log
        print(f"OpenAI API returned an API Error: {e}")
        
    except openai.error.APIConnectionError as e:
        #Handle connection error here
        print(f"Failed to connect to OpenAI API: {e}")
        
    except openai.error.RateLimitError as e:
        #Handle rate limit error (we recommend using exponential backoff)
        print(f"OpenAI API request exceeded rate limit: {e}")
    
    return cleaned_arguments

In [13]:
sents = '''유진이는 부산에서 태어났다. 왜냐하면 부산에서 태어난 사람은 일반적으로 대한민국 국민일 것이기 때문이다.
다음과 같은 법조항과 규정들을 근거로(국적법). 그래서 그의 부모가 외국인이었거나 그가 귀화한 미국인이 된 경우가 아니라면 유진이는 한국인이다.'''

In [14]:
inspect_data(sents)

Original: 유진이는 부산에서 태어났다. 왜냐하면 부산에서 태어난 사람은 일반적으로 대한민국 국민일 것이기 때문이다.
다음과 같은 법조항과 규정들을 근거로(국적법). 그래서 그의 부모가 외국인이었거나 그가 귀화한 미국인이 된 경우가 아니라면 유진이는 한국인이다.
주장: 유진이는 한국인이다.
반박: 없음
자료: 유진이는 부산에서 태어났다.
보장: 부산에서 태어난 사람은 일반적으로 대한민국 국민일 것이다.
뒷받침: 없음
제한 조건: 없음
점수: 2
피드백: 주장과 자료가 포함되어 있지만, 보장과 뒷받침이 명시적으로 나타나지 않았습니다. 또한 반박이 없으므로 더 강력한 논증을 위해 추가적인 정보나 반박을 고려해 볼 수 있습니다.


{'claim': '유진이는 한국인이다.',
 'rebuttal': '',
 'data': '유진이는 부산에서 태어났다.',
 'warrant': '부산에서 태어난 사람은 일반적으로 대한민국 국민일 것이다.',
 'back': '',
 'qualify': '',
 'score': 2,
 'feedback': '주장과 자료가 포함되어 있지만, 보장과 뒷받침이 명시적으로 나타나지 않았습니다. 또한 반박이 없으므로 더 강력한 논증을 위해 추가적인 정보나 반박을 고려해 볼 수 있습니다.'}

## 네이버 뉴스 기사를 불러와서 질문하기

In [17]:
url = 'https://n.news.naver.com/mnews/article/366/0000923133?sid=101'

In [18]:
import requests
from bs4 import BeautifulSoup

def summarize(url=None, text=None):
    if (url is None) and (text is None):
        print('URL 또는 분석할 텍스트를 입력해 주세요.')
        return
    
    # 현재는 네이버 뉴스 기사만 가지고 옴
    if url is not None:
        req = requests.get(url)
        soup = BeautifulSoup(req.text, "html.parser")
        text = soup.select_one('article#dic_area.go_trans._article_content').text
        text = text.strip()
    
    # 메시지 설정하기
    query = f"주어진 텍스트의 내용을 요약해 줘. 텍스트:{text}"
    messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": query}
    ]

    # ChatGPT API 호출하기
    response = openai.ChatCompletion.create(model=MODEL, messages=messages, temperature=TEMPERATURE)
    answer = response.choices[0].message['content']
    answer = answer.strip()

    print('입력된 텍스트')
    print(text)
    print('요약:')
    print(answer)
    
    return answer

In [19]:
summarize(url)

입력된 텍스트
유안타證 “20분 만에 조정 완료…패닉 투매 아냐”        연일 증시를 뜨겁게 달궜던 초전도체주가 급락한 가운데, 전날 초전도체 종목이 일제히 하락한 것이 알고리즘 매매로 의심된다는 분석이 나왔다.9일 유안타증권은 ‘손바뀜도 초전도더라 - 제2의 시타델 DMA 사태 우려’라는 제목의 보고서를 내고 이런 내용을 발표했다. '



유안타증권 제공.        고경범 유안타증권 연구원은 “전날 초전도체 관련주는 LK-99가 초전도체가 아니라는 메릴랜드 CMTC의 발표 뉴스로 급락했는데, 주목할 것은 조정보다 ‘진행된 시간’이다”라고 평가했다. 그는 “초전도체 관련주의 조정과 거래량 증가는 2시 이후부터 사실상 20분 만에 완료됐다고 볼 수 있다”라고 분석했다.이어 그는 “특히 2시 12분 경의 매도 주문이 주가 하락에 결정적이었는데, 7거래일 동안 회자된 이슈인 데다가 다수의 개인 투자자 분포를 감안하면 8분의 조정 시간은 극히 짧다”라고 했다.이런 현상을 ‘공황 매도(패닉셀)’로 평가하기에는 한계가 있다고 고 연구원은 설명했다. 그는 “해당 테마로 시세를 견인한 기존 매수자의 매도로 보는 것이 합리적인데, 해당 종목들 대부분은 펀드 기반의 기관과 외국인이 투자하기에는 작은 사이즈의 종목이고 공매도 또한 제한적이기 때문”이라면서 “관련주에서 공통으로 발견되는 짧은 시간의 거래량 폭증과 호가 하락에서 알고리즘 매매와 주로 사용되는 직접시장접근(DMA, Direct Market Acess) 채널 거래가 의심된다”라고 했다.이어 고 연구원은 지난 2017~2018년 발생한 미국 시타델증권의 DMA를 이용한 알고리즘 매매 사례를 언급했다. 지난 2017년 10월부터 2018년 5월까지 미국의 대형 헤지펀드 시타델 계열사 시타델증권은 메릴린치증권 서울 지점을 통해 264종목에 대해 6786회 거래를 한 바 있다. 당시 금융 당국은 시타델증권이 시장 질서를 교란했다고 보고 올해 1월 100억원대 과징금을 부과했다.이들은 컴퓨터가 짧은 시간에 많은 주문을

'유안타증권은 초전도체주의 급락은 알고리즘 매매로 의심된다는 분석을 내놓았다. 초전도체 관련주의 조정은 20분 만에 완료되었으며, 이는 패닉셀로 평가하기에는 짧은 시간이라고 분석되었다. 이러한 현상은 알고리즘 매매와 직접시장접근(DMA) 채널 거래의 의심을 받고 있다. 이와 관련하여 2017~2018년에 발생한 미국 시타델증권의 DMA를 이용한 알고리즘 매매 사례를 언급하였다. 또한, 최근 시장 상황에서도 알고리즘 매매로 의심되는 경우가 자주 보인다고 설명하였다. 이에 따라 당국은 행정적 조치에 대해 고려해야 할 시점이라고 언급하였다. 초전도체주의 급등은 LK-99라는 초전도체 물질에 대한 논문이 발표된 것으로 시작되었으나, 미국의 한 연구소가 LK-99가 초전도체가 아니라고 밝히자, 관련주는 일제히 급락하였다.'

## 특정 PDF를 읽어들이고 이에 대해 질문하기

In [20]:
from PyPDF2 import PdfReader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.summarize import load_summarize_chain
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

In [21]:
# 내가 읽고 싶은 파일의 위치를 지정하도록 한다.
reader = PdfReader("personal/the scientific method as myth and ideal.pdf")

raw_text = ""

for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [23]:
llm = OpenAI(temperature=0, openai_api_key = OPENAI_API_KEY)

In [24]:
summary_chain = load_summarize_chain(llm, chain_type="map_reduce")
summarize_document_chain = AnalyzeDocumentChain(combine_docs_chain=summary_chain)

In [25]:
summarize_document_chain.run(raw_text)

' This article examines the shortcomings of the popularly-taught "Scientific Method" and provides corrections to common misconceptions. It looks at the lack of consensus on the method, with different views from Aristotle, Bacon, and Newton, and discusses the classical inductivism view. It also considers the implications for introductory pedagogical contexts, the importance of social dynamics for the scientific enterprise, the role of scientific communities, and the aims, values, and virtues of scientific activity. It suggests that the myth of the scientific method should be discarded, but the ideals associated with it should be promoted.'

## 여러 개의 파일에 대한 분석

In [26]:
loader = PyPDFLoader("personal/the scientific method as myth and ideal.pdf")
pages = loader.load_and_split()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100) #chunk overlap seems to work better
documents = text_splitter.split_documents(pages)

In [27]:
print (f'You have {len(documents)} document(s) in your data')

You have 29 document(s) in your data


In [30]:
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', max_retries=20, openai_api_key = OPENAI_API_KEY)
vectorstore = Chroma.from_documents(documents, embeddings)
model = ChatOpenAI(temperature=TEMPERATURE, model_name=MODEL, openai_api_key = OPENAI_API_KEY)
qa = ConversationalRetrievalChain.from_llm(model, vectorstore.as_retriever())

In [31]:
def ask(question):
    result = qa({"question":question, "chat_history":[]})
    print(result['answer'])
    
    return result['answer']

In [32]:
ask('논문의 핵심 주장이 뭐야?')

논문의 핵심 주장은 "The Scientiﬁc Method"라는 개념은 과학적 활동과 추론의 정확한 표현이 아니며, 역사적으로도 현대적으로도 합의된 내용이 없다는 것입니다. 이러한 개념은 과학적 방법론에 대한 오해와 함께 자주 소개되는 오류들을 포함하고 있으며, 이러한 오류들은 과학 교육에 있어서 어떤 결론을 도출해야 하는지에 대한 고려도 포함하고 있습니다.


'논문의 핵심 주장은 "The Scientiﬁc Method"라는 개념은 과학적 활동과 추론의 정확한 표현이 아니며, 역사적으로도 현대적으로도 합의된 내용이 없다는 것입니다. 이러한 개념은 과학적 방법론에 대한 오해와 함께 자주 소개되는 오류들을 포함하고 있으며, 이러한 오류들은 과학 교육에 있어서 어떤 결론을 도출해야 하는지에 대한 고려도 포함하고 있습니다.'

## 여러 개의 논문을 대상으로 한 분석

In [34]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [35]:
persist_directory = 'wos_gifted_db'
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002', max_retries=20, openai_api_key = OPENAI_API_KEY)
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [37]:
model = ChatOpenAI(temperature=TEMPERATURE, model_name=MODEL, openai_api_key = OPENAI_API_KEY)
qa = ConversationalRetrievalChain.from_llm(model, vectorstore.as_retriever(search_kwargs={"k": 3}), return_source_documents=True)

In [40]:
chat_history = []

In [39]:
def manage_history(lists):    
    tot_size = 0
    if len(lists) >= 1:
        for item in lists:
            tot_size += len(item.content)

        if tot_size >= 3000:
            lists = lists[2:]  # chat_history 전역 변수를 수정
        
        return lists
    else:
        return lists
        
def questioning(lists, query):
    lists = manage_history(lists)
    
    result = qa({"question": query, "chat_history": lists})
    print(result['answer']) # 응답 출력
        
    #Reference 출력
    refs = []
    print('Reference:')
    for item in result['source_documents']:
        filename_with_extension = os.path.basename(item.metadata['source'])
        filename = os.path.splitext(filename_with_extension)[0]
        print(filename)
        refs.append(filename)
        
    lists.append(HumanMessage(content=query, additional_kwargs={}, example=False))
    lists.append(AIMessage(content=result['answer'], additional_kwargs={'source':refs}, example=False))
        
    return lists

In [41]:
chat_history = questioning(chat_history, "과학영재를 선발할 때 기준은 무엇일까?")

과학영재를 선발할 때 기준은 국가나 기관에 따라 다를 수 있습니다. 일반적으로는 학업 성적, 과학적 업적, 창의력, 문제 해결 능력, 실험 및 연구 경험 등이 고려되는 경우가 많습니다. 또한, 시험이나 면접 등의 절차를 거쳐 선발되기도 합니다. 선발 기준은 각 기관이나 국가의 정책에 따라 다를 수 있으므로, 구체적인 정보를 얻기 위해서는 해당 기관이나 국가의 공식 웹사이트나 관련 담당자에게 문의하는 것이 좋습니다.
Reference:


In [42]:
chat_history = questioning(chat_history, "과학영재 교육은 몇 살 때부터 시작하는 것이 좋을까?")

과학영재 교육은 일반적으로 초등학교 중학년부터 시작하는 것이 좋습니다. 이 시기에는 학생들이 기초적인 과학 개념을 이해하고 실험을 통해 문제 해결 능력을 키울 수 있습니다. 그러나 어린이들이 흥미를 가지고 자발적으로 과학에 대해 탐구하는 경우, 더 일찍 시작할 수도 있습니다. 따라서 학생의 흥미와 준비 상태를 고려하여 결정하는 것이 중요합니다.
Reference:


In [43]:
chat_history = questioning(chat_history, "과학영재를 선발할 때 고려해야 하는 조건은 무엇일까?")

과학영재를 선발할 때 고려해야 하는 조건은 다양할 수 있습니다. 일반적으로는 다음과 같은 요소들이 고려될 수 있습니다:

1. 학업 성적: 과학 분야에서의 학업 성적이 좋은지 여부를 평가할 수 있습니다.
2. 창의성과 독창성: 과학적 문제 해결에 대한 창의적인 사고와 독창성을 가지고 있는지 여부를 평가할 수 있습니다.
3. 실험 능력: 실험 설계와 데이터 분석 등 실험 능력을 평가할 수 있습니다.
4. 과학적 호기심: 과학에 대한 호기심과 탐구 정신을 가지고 있는지 여부를 평가할 수 있습니다.
5. 협력과 팀워크: 과학적인 프로젝트나 연구에서 협력과 팀워크를 잘 할 수 있는지 여부를 평가할 수 있습니다.
6. 수상 경력: 과학 분야에서의 수상 경력이 있는지 여부를 평가할 수 있습니다.

이 외에도 학교나 기관마다 선발 조건이 다를 수 있으므로, 구체적인 선발 기준은 해당 학교나 기관의 지침을 참고해야 합니다.
Reference:
